In [138]:
!pip install pyspark
!pip install findspark

In [9]:
import os
import pyspark
from pyspark.sql import SparkSession

In [74]:
import findspark

try:
    findspark.init()
except:
    print('아래 방법으로 path 재설정')

In [75]:
try:
    print(os.environ['SPARK_HOME'])
except:
    os.environ['SPARK_HOME'] = 'C:\spark-3.0.0-preview2-bin-hadoop2.7\spark-3.0.0-preview2-bin-hadoop2.7'
    print(os.environ['SPARK_HOME'])
try:
    print(os.environ['JAVA_HOME'])
except KeyError:
    os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_231'
    print(os.environ['JAVA_HOME'])
print(os.environ['PATH'])

C:\spark-3.0.0-preview2-bin-hadoop2.7\spark-3.0.0-preview2-bin-hadoop2.7
C:\Program Files\Java\jdk1.8.0_231
C:\Users\Namsik\Anaconda3;C:\Users\Namsik\Anaconda3\Library\mingw-w64\bin;C:\Users\Namsik\Anaconda3\Library\usr\bin;C:\Users\Namsik\Anaconda3\Library\bin;C:\Users\Namsik\Anaconda3\Scripts;C:\Users\Namsik\Anaconda3\bin;C:\Users\Namsik\Anaconda3\condabin;C:\Program Files\Rockwell Software\RSCommon;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.1\libnvvp;C:\windows\system32;C:\windows;C:\windows\System32\Wbem;C:\windows\System32\WindowsPowerShell\v1.0;C:\windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Program Files\NVIDIA Corporation\Nsight Compute 2019.4.0;";C:\Users\Namsik\AppData\Local\Microsoft\WindowsApps\";C:\Program Files\Git\cmd;C:\Program Files\Intel\WiFi\bin;C:\Pro

In [76]:
findspark.init()

In [77]:
if 'spark_data' in os.listdir():pass
else:
    import wget
    import zipfile
    wget.download('https://www.dropbox.com/sh/5c9daz43f5lstzd/AAAPN_iMkKzgqJ1k3tml7FNHa?dl=1')
    with zipfile.ZipFile('spark_data.zip', 'r') as zip_ref:
        zip_ref.extractall('spark_data')

In [78]:
"""
스파크 애플리케이션은 SparkSession이라 불리는 드라이버 프로세스로 제어합니다. 
SparkSession 인스턴스는 사용자가 정의한 처리 명령을 클러스터에서 실행합니다. 
하나의 SparkSession은 하나의 스파크 애플리케이션에 대응합니다. 
스칼라와 파이썬 콘솔을 시작하면 spark변수로 SparkSession을 사용할 수 있습니다.
"""
spark = SparkSession.builder.master("local")\
.getOrCreate()

In [79]:
"""
일정범위의 숫자를 만드는 작업. 
이 숫자들은 스프레드시트에서 컬럼명을 지정한것과 같음.
"""
myRange = spark.range(1000).toDF('number')

In [80]:
"""
방금 생성한 DataFrame(MyRange)는 한개의 컬럼과 1000개의 로우로 구성되며 각 로우에는 0부터999까지의 값이 할당됨.
이 숫자들은 분산 컬렉션을 나타내며 클러스터 모드에서 코드예제를 실행하면 숫자 범위의 각 부분이 서로 다른 익스큐터에 할당됨.
이것이 스파크의 DataFrame임.
"""
myRange

DataFrame[number: bigint]

In [81]:
myRange.head()

Row(number=0)

In [82]:
myRange.head(10)

[Row(number=0),
 Row(number=1),
 Row(number=2),
 Row(number=3),
 Row(number=4),
 Row(number=5),
 Row(number=6),
 Row(number=7),
 Row(number=8),
 Row(number=9)]

In [83]:
"""
스파크의 핵심 데이터 구조는 불변성을 가짐. 
한번 생성하면 변경할 수 없으며 변경을 하려면 변경 방법을 스파크에 알려줘야함.
이때 사용하는 명령을 트렌스포메이션이라 하며 아래 코드는 짝수를 찾는 간단한 트랜스포메이션 예제
"""
divisBy2 = myRange.where('number % 2 = 0')
divisBy2

DataFrame[number: bigint]

In [84]:
divisBy2.head()

Row(number=0)

In [85]:
divisBy2.head(10)

[Row(number=0),
 Row(number=2),
 Row(number=4),
 Row(number=6),
 Row(number=8),
 Row(number=10),
 Row(number=12),
 Row(number=14),
 Row(number=16),
 Row(number=18)]

In [86]:
divisBy2.count(),myRange.count()

(500, 1000)

In [87]:
"""
스파크는 다양한 데이터소스를 지원함.
아래 예제에서는 스파크 DataFrame의 스키마 정보를 알아내는 스키마 추론기능을 사용함.
(스키마 = 데이터의 컬럼 및 컬럼의 데이터 유형에 대한 정보)
그리고 파일의 첫 로우를 헤더로 지정하는 옵션도 설정
"""
flightData2015 = spark.read.option('inferSchema','true').option('header','true')\
.csv('spark_data/flight-data/csv/2015-summary.csv')

In [88]:
"""
take 액션을 호출하면 head와 같은 결과를 확인할 수 있음
해당 DataFrame은 ['DEST_COUNTRY_NAME','ORIGHIN_COUNTRY_NAME','count']라는 컬럼을 가짐
"""
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [89]:
flightData2015.head(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [90]:
"""
정수 데이터 타입인 count 컬럼을 기준으로 explain액션을 이용해 데이터를 정렬.
sort는 호출시 데이터에 아무런 변화도 일어나지않음.
아래 호출된 내용은 실행계획으로 디버깅과 스파크의 실행과정을 이해하는데 도움을 주는 도구일 뿐임
"""
flightData2015.sort('count').explain()

== Physical Plan ==
*(2) Sort [count#238 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#238 ASC NULLS FIRST, 5)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#236,ORIGIN_COUNTRY_NAME#237,count#238] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/D:/Dropbox/spark_data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [91]:
"""
앞서 말한것과같이 sort를 한다해도 바뀌진않고 실행계획을 실행하여야함
"""
flightData2015.head(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [92]:
"""
트랜스포메이션 실행 계획을 시작하기위해선 액션을 호출해야하며 이를 위해 몇가지 설정이 필요함.
스파크는 셔플 수행 시 기본적으로 200개의 셔플 파티션을 생성하며 이 값을 5로 설정해 셔플의 출력 파티션 수를 줄임
"""
spark.conf.set('spark.sql.shuffle.partitions','5')

In [93]:
flightData2015.head(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [94]:
flightData2015.sort('count').take(5)

[Row(DEST_COUNTRY_NAME='Malta', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='Saint Vincent and the Grenadines', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Gibraltar', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1)]

In [95]:
"""
스파크는 언어에 상관없이 같은 방식으로 트랜스포메이션을 실행할 수 있음.
사용자가 SQL이나 DataFrame으로 비즈니스 로직을 표현하면 스파크에서 실제 코드를 실행하기 전에 그 로직을 기본 실행 계획으로 컴파일함.
스파크 SQL을 사용하면 모든 DataFrame을 테이블이나 뷰로 등록한 후 SQL 쿼리를 사용할 수 있음.
아래 코드로 createOrReplaceTempView 메서드를 호출하면 모든 DataFrame을 테이블이나 뷰로 만들 수 있음
"""
flightData2015.createOrReplaceTempView('flight_data_2015')

In [96]:
"""
이제 SQL로 데이터를 조회할 수 있으며 spark.sql 메서드로 SQL 쿼리를 실행함
DataFrame에 쿼리를 수행하면 새로운 DataFrame을 반환함.
로직을 작성할 때 반복적인 느낌이 들지만 효율적인 방식임
아래처럼 sql문으로 수행할 수도있고, DataFrame의 내장함수로도 가능

아래 코드는 DataFrame에서 DEST_COUNTRY_NAME 컬럼의 나라 별 로우 갯수를 출력
(출발 국가별 로우 갯수)
"""

sqlWay = spark.sql("SELECT DEST_COUNTRY_NAME, count(1) FROM flight_data_2015 GROUP BY DEST_COUNTRY_NAME")
dataFrameWay = flightData2015.groupBy("DEST_COUNTRY_NAME").count()

sqlWay.explain()
print('-'*100)
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#236], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#236, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#236], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#236] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/D:/Dropbox/spark_data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
----------------------------------------------------------------------------------------------------
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#236], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#236, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#236], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#236] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/D:/Dropbox/spark_data/flight-data/csv/2015-summary.csv], Partition

In [97]:
sqlWay.head(5)

[Row(DEST_COUNTRY_NAME='Moldova', count(1)=1),
 Row(DEST_COUNTRY_NAME='Bolivia', count(1)=1),
 Row(DEST_COUNTRY_NAME='Algeria', count(1)=1),
 Row(DEST_COUNTRY_NAME='Turks and Caicos Islands', count(1)=1),
 Row(DEST_COUNTRY_NAME='Pakistan', count(1)=1)]

In [98]:
dataFrameWay.head(5)

[Row(DEST_COUNTRY_NAME='Moldova', count=1),
 Row(DEST_COUNTRY_NAME='Bolivia', count=1),
 Row(DEST_COUNTRY_NAME='Algeria', count=1),
 Row(DEST_COUNTRY_NAME='Turks and Caicos Islands', count=1),
 Row(DEST_COUNTRY_NAME='Pakistan', count=1)]

In [99]:
sqlWay.count(),dataFrameWay.count()

(132, 132)

In [100]:
"""
번외. pandas 문법으로 시도

똑같음
"""
import pandas as pd
df = pd.read_csv('spark_data/flight-data/csv/2015-summary.csv')
pandasWay = df.groupby('DEST_COUNTRY_NAME').count()
pandasWay.head()

,ORIGIN_COUNTRY_NAME,count
DEST_COUNTRY_NAME,,
Algeria,1,1
Angola,1,1
Anguilla,1,1
Antigua and Barbuda,1,1
Argentina,1,1


In [101]:
"""
스파크는 빅데이터 문제를 빠르게 해결하는데 필요한 수백개의 함수를 제공.
다음 예제에서는 특정 위치를 왕래하는 최대 비행횟수를 구하기 위해 max함수를 사용.
max 함수는 DataFrame의 특정 컬럼값을 스캔하면서 이전 최댓값보다 더 큰 값을 찾음.
max 함수는 필터링을 수행해 단일 로우를 결과로 반환하는 트랜스포메이션임.
"""

from pyspark.sql.functions import max

print(flightData2015.select(max('count')).take(1))
print(flightData2015.select(max('count')).take(2))

[Row(max(count)=370002)]
[Row(max(count)=370002)]


In [102]:
"""
위의 코드처럼 max는 단일 로우를 반환하기에 take에 무슨 숫자를 넣든 같은 단일 로우를 반환.
이번엔 상위 5개의 도착 국가를 찾아내는 복잡한 코드를 실행.
"""

maxSql = spark.sql("SELECT DEST_COUNTRY_NAME, sum(count) as destinaton_total\
 FROM flight_data_2015\
 GROUP BY DEST_COUNTRY_NAME\
 ORDER BY sum(count) DESC\
 LIMIT 5")

maxSql.show()

+-----------------+----------------+
|DEST_COUNTRY_NAME|destinaton_total|
+-----------------+----------------+
|    United States|          411352|
|           Canada|            8399|
|           Mexico|            7140|
|   United Kingdom|            2025|
|            Japan|            1548|
+-----------------+----------------+



In [103]:
"""
이번엔 sql구문이 아닌 DataFrame 구문
"""

from pyspark.sql.functions import desc

flightData2015.groupBy('DEST_COUNTRY_NAME').sum('count')\
.withColumnRenamed('sum(count)','destination_total')\
.sort(desc('destination_total'))\
.limit(5).show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [104]:
"""
번외. 위의 코드를 뜯어보자
"""
first_action = flightData2015.groupBy('DEST_COUNTRY_NAME').sum('count')
first_action.show(5)
print('='*50)
second_action = first_action.withColumnRenamed('sum(count)','destination_total')
second_action.show(5)
print('='*50)
third_action = second_action.sort(desc('destination_total'))
third_action.show(5)
print('='*50)
print('limit(5)와 show(5)는 출력상으로는 같은 의미이지만 limit(5)는 dataframe자체를 변환')
fourth_action = third_action.limit(5)
fourth_action.show()

+--------------------+----------+
|   DEST_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|             Moldova|         1|
|             Bolivia|        30|
|             Algeria|         4|
|Turks and Caicos ...|       230|
|            Pakistan|        12|
+--------------------+----------+
only showing top 5 rows

+--------------------+-----------------+
|   DEST_COUNTRY_NAME|destination_total|
+--------------------+-----------------+
|             Moldova|                1|
|             Bolivia|               30|
|             Algeria|                4|
|Turks and Caicos ...|              230|
|            Pakistan|               12|
+--------------------+-----------------+
only showing top 5 rows

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|        